In [8]:
import numpy as np
import librosa
import soundfile as sf

def detect_nonsilence_regions(filename, silence_threshold=0.05, min_duration=0.15, target_duration=0.3, sr=None):
    # Load the audio file
    y, sr = librosa.load(filename, sr=sr)

    # Detect non-silent regions
    non_silent_samples = librosa.effects.split(y, top_db=silence_threshold)

    # Filter out regions shorter than min_duration
    min_samples = int(min_duration * sr)
    non_silent_samples = [interval for interval in non_silent_samples if interval[1] - interval[0] >= min_samples]

    # Adjust start and end points to make duration equal to target_duration
    target_samples = int(target_duration * sr)
    adjusted_samples = []
    for start, end in non_silent_samples:
        diff = target_samples - (end - start)
        start -= diff // 2
        end += diff // 2

        # Handle cases where start or end goes out of bounds
        start = max(0, start)
        end = min(len(y), end)

        adjusted_samples.append((start, end))

    return adjusted_samples

def extract_and_save_regions(filename, output_prefix, regions, sr=None):
    y, _ = librosa.load(filename, sr=sr)
    for i, (start, end) in enumerate(regions):
        sf.write(f"{output_prefix}_region_{i}.wav", y[start:end], sr)

if __name__ == "__main__":
    filename = "HackRF_20230920_152800Z_433000kHz_IQ.wav"
    regions = detect_nonsilence_regions(filename)
    extract_and_save_regions(filename, "output", regions)


In [24]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import os

def trim_audio_to_duration(file_path, output_path, desired_duration=300, min_silence_len=100, silence_thresh=-50):
    """
    Trims silence from the start and end of an audio file and ensures the resulting file does not exceed the desired duration.
    Prioritizes retaining the latter part of the audio signal if the non-silent duration exceeds the desired length.
    
    Parameters:
    - file_path: Path to the WAV file to process.
    - output_path: Path where the trimmed audio will be saved.
    - desired_duration: Maximum duration of the resulting audio file after trimming (in ms).
    - min_silence_len: Minimum length of silence to be used for detection (in ms).
    - silence_thresh: Silence threshold (in dBFS).
    """
    audio = AudioSegment.from_wav(file_path)
    
    # Detect nonsilent parts
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    
    # If no nonsilent ranges are detected, the audio is probably all silence.
    if not nonsilent_ranges:
        raise ValueError("No non-silent segments detected in the audio.")
    
    # Only consider the start of the first non-silent segment and the end of the last non-silent segment.
    start, _ = nonsilent_ranges[0]
    _, end = nonsilent_ranges[-1]
    signal = audio[start:end]

    # If signal length exceeds the desired_duration, truncate from the start to prioritize the latter part
    if len(signal) > desired_duration:
        signal = signal[-desired_duration:]
    
    # Save the trimmed audio
    signal.export(output_path, format="wav")

def process_folder(source_folder, output_folder):
    """
    Processes all WAV files in the source folder and saves the trimmed files in the output folder.
    
    Parameters:
    - source_folder: Path to the folder containing WAV files.
    - output_folder: Path to the folder where trimmed files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file_name in os.listdir(source_folder):
        if file_name.endswith('.wav'):
            source_file_path = os.path.join(source_folder, file_name)
            trimmed_file_name = "trimmed_" + file_name
            output_file_path = os.path.join(output_folder, trimmed_file_name)
            trim_audio_to_duration(source_file_path, output_file_path)

# Usage
input_directory = r"C:\Users\CAN-LAB\Desktop\test"
output_directory = r"C:\Users\CAN-LAB\Desktop\test(Trimmed)"
process_folder(input_directory, output_directory)

In [26]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import os

def trim_audio(file_path, output_path, min_silence_len=500, silence_thresh=-20, desired_duration=300):
    """
    Trims silence from an audio file and ensures the resulting file has a fixed desired duration.
    
    Parameters:
    - file_path: Path to the WAV file to process.
    - output_path: Path where the trimmed audio will be saved.
    - min_silence_len: Minimum length of silence to be used for detection (in ms).
    - silence_thresh: Silence threshold (in dBFS).
    - desired_duration: Desired duration of the resulting audio file after trimming (in ms).
    """
    audio = AudioSegment.from_wav(file_path)
    
    # Detect nonsilent parts
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    
    # Concatenate nonsilent parts to get the signal
    signal = AudioSegment.empty()
    for start, end in nonsilent_ranges:
        signal += audio[start:end]
    
    # Calculate the padding needed to center the signal in the desired duration
    total_padding = desired_duration - len(signal)
    start_padding = total_padding // 2
    end_padding = total_padding - start_padding

    start_silence = AudioSegment.silent(duration=start_padding)
    end_silence = AudioSegment.silent(duration=end_padding)
    
    # Concatenate silent segments with the signal
    trimmed_signal = start_silence + signal + end_silence
    
    # Save the trimmed audio
    trimmed_signal.export(output_path, format="wav")

def process_folder(source_folder, output_folder):
    """
    Processes all WAV files in the source folder and saves the trimmed files in the output folder.
    
    Parameters:
    - source_folder: Path to the folder containing WAV files.
    - output_folder: Path to the folder where trimmed files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file_name in os.listdir(source_folder):
        if file_name.endswith('.wav'):
            source_file_path = os.path.join(source_folder, file_name)
            trimmed_file_name = file_name.replace('.wav', ' (trimmed).wav')
            output_file_path = os.path.join(output_folder, trimmed_file_name)
            trim_audio(source_file_path, output_file_path)

# Usage
source_folder = r"C:\Users\CAN-LAB\Desktop\test"
output_folder = r"C:\Users\CAN-LAB\Desktop\test(Trimmed)"
process_folder(source_folder, output_folder)
